<a id="setup"></a>
## LangChain memory with watsonx.ai models

This notebook previously illustrated how to use *ConversationBufferMemory* with models included in watsonx.ai.

As of the v0.3 release of LangChain, it is recommended that LangChain users take advantage of LangGraph persistence to incorporate memory into new LangChain applications. LangGraph is a library for building stateful, multi-actor applications with LLMs, used to create agent and multi-agent workflows.

This notebook contains sample code for using LangGraph to model the deprecated *ConversationChain* (a chain impementing a conversation and loading context from memory) from LangChain. This is the LangChain recommended migration path for migrating from ConversationChain.


### Install dependecies

In [ ]:
# Install the required LangChain and IBM packages
!pip install ibm-watson-machine-learning --upgrade
!pip install langchain | tail -n 1
!pip install langgraph
!pip install langchain-community
!pip install -U langchain-ibm

### Define the WML credentials
This cell defines the WML credentials required to work with watsonx Foundation Model inferencing.

**Action:** Provide the IBM Cloud user API key. For details, see
[documentation](https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui).

In [ ]:
import getpass

credentials = {
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey": getpass.getpass("Please enter your WML api key (hit enter): ")
}

url = credentials['url']
apikey = credentials['apikey']

### Define the project id
The Foundation Model requires project id that provides the context for the call. We will obtain the id from the project in which this notebook runs. Otherwise, please provide the project id.

In [ ]:
import os

try:
    project_id = os.environ["PROJECT_ID"]
except KeyError:
    project_id = input("Please enter your project_id (hit enter): ")

### Import required packages

In [ ]:
# Import packages from IBM
from ibm_watson_machine_learning.foundation_models import Model
from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
from ibm_watson_machine_learning.foundation_models.utils.enums import DecodingMethods
from langchain_ibm import WatsonxLLM

# Import packages from langchain
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.llms.utils import enforce_stop_tokens

In [ ]:
# If you want to change model types, you can look them up with this line
print([model.name for model in ModelTypes])

### Create the LangChain model object

In [ ]:
# We will use the llama model

# Experiment with different models. We noticed that Flan produces a more concise ouput, but llama - more descriptive and better quality#model_id_1 = "meta-llama/llama-2-70b-chat"
# model_id_1 = "google/flan-ul2"
model_id_1 = "meta-llama/llama-3-1-70b-instruct"

model1_parameters = {
    "decoding_method": "greedy",
    "max_new_tokens": 100,
    "min_new_tokens": 1,
    "top_k": 50,
    "top_p": 1
}

# WML model objects - will be used to create models for the LangChain API
# Create the model objects that will be used by LangChain

current_llm = WatsonxLLM(
    model_id=model_id_1,
    url=url,
    project_id=project_id,
    params=model1_parameters,
    apikey=apikey)

In [ ]:
# Universally Unique Identifiers (UUIDs) provide a convenient way to 
# generate unique IDs and random strings in Python
import uuid

# Import the required langgraph libraries to manage memory
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# Use the model available from IBM that was created in th eprevious cell
model = current_llm

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)

# Define the function that calls the model
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}

# Define the two nodes we will cycle between
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
# The MemorySaver() checkpoint saver stores checkpoints in memory using a defaul dictionary.
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

# The thread id is a unique key that identifies
# this particular conversation.
# We'll just generate a random uuid here.
thread_id = uuid.uuid4()
config = {"configurable": {"thread_id": thread_id}}

In [ ]:
# Example of a first quesiton from the user
query = "From the following customer complaint, extract 3 factors that caused the customer to be unhappy. \
                            Put each factor on a new line."
input_messages = [
    {
        "role": "system",
        "content": "Customer complaint: I am writing you this statement to delete the \
                            following information on my credit report. The items I need deleted are listed in the report. \
                            I am a victim of identity thief, I demand that you remove these errors to correct my report immediately! \
                            I have reported this to the federal trade commission and have attached the federal trade commission affidavit. \
                            Now that I have given you the following information, please correct my credit report or I shall proceed with involving my attorney! \
                            Numbered list of complaints:",
    },
    {"role": "user", "content": query},
]
for event in app.stream({"messages": input_messages}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()

In [ ]:
# Example of a second question from the user
query = "Does the numbered list of complaints contain a statement about identity fraud? Provide a short answer: yes or no."

input_messages = [{"role": "system", "content": query}, {"role": "user", "content": query},]
for event in app.stream({"messages": input_messages}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()

### Authors: 
 **Elena Lowery**, Data and AI Architect
### Updated by:
**Andre de Waal**, Learning Content Developer